In [24]:
# Imports
import torch.nn as nn
import torch

In [36]:
# Task 0. Generator
class Generator(nn.Module):
    """
    Generator subclass for the generator side of the network
    """
    def __init__(self,
                 input_size,
                 hidden_size,
                 output_size):
        """
        Initialize Generator class

        Args:
            input_size - integer size of the input tensor
            hidden_size - integer size of the hidden layer
            output_size - integer size of the output layer
        """
        super().__init__()
        self.main = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, output_size)
        )

    def forward(self, x):
        """
        Forward pass of the Generator network

        Args:
            x - Input tensor of shape (batch_size, input_size)

        Returns:
            Output tensor of shape (batch_size, output_size)
        """
        return self.main(x)


In [37]:
# 0-main
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
genTester = Generator

G_Test = genTester(1,1,1)

print(G_Test)

Generator(
  (model): Sequential(
    (0): Linear(in_features=1, out_features=1, bias=True)
    (1): Tanh()
    (2): Linear(in_features=1, out_features=1, bias=True)
    (3): Tanh()
    (4): Linear(in_features=1, out_features=1, bias=True)
  )
)


In [27]:
# Task 1. Discriminator
class Discriminator(nn.Module):
    """
    Discriminator subclass for the discriminator side of the network
    """
    def __init__(self,
                 input_size,
                 hidden_size,
                 output_size):
        """
                Initialize Discriminator class

        Args:
            input_size - integer size of the input tensor
            hidden_size - integer size of the hidden layer
            output_size - integer size of the output layer
        """
        super().__init__()

        self.main = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.Sigmoid(),
            nn.Linear(hidden_size, hidden_size),
            nn.Sigmoid(),
            nn.Linear(hidden_size, output_size),
            nn.Sigmoid()
        )

    def forward(self, x):
        """
        Forward pass of the Discriminator network

        Args:
            x - Input tensor of shape (batch_size, input_size)

        Returns:
            Output tensor of shape (batch_size, output_size)
        """
        return self.main(x)

In [28]:
# 1-main
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

disTester = Discriminator

D_Test = disTester(1,1,1)

print(D_Test)

Discriminator(
  (main): Sequential(
    (0): Linear(in_features=1, out_features=1, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=1, out_features=1, bias=True)
    (3): Sigmoid()
    (4): Linear(in_features=1, out_features=1, bias=True)
    (5): Sigmoid()
  )
)


In [29]:
# Task 2. Sample Z
def sample_Z(mu, sigma, sampleType, size=(1, 1)):
    """
    Creates input for the generator and discriminator

    Added size argument to sample_Z function. Per torch.randn and torch.normal
        documentataion, size is a sequence of integers defining the shape of
        the output tensor.

    Args:
        mu -  The mean of the distribution
        sigma - The standard deviation of the distribution
        sampleType - A variable that selects which model to sample for
            The variable should accept a G or D as string values
        size - tuple of ints defining the shape of the output tensor

    Returns:
        a torch.Tensor type for both generator and discriminator if the
            parameters are correct, otherwise return 0
    """
    # For the generator, find a random number from torch.rand
    # Multiply that number by sigma and add mu
    # From the task, input data for the generator should be random
    if sampleType == "G":
        return torch.randn(size=size)
    # For the discriminator, take a number from the normal distribution
    # using torch.normal
    elif sampleType == "D":
        return torch.normal(mean=mu, std=sigma, size=size)
    else:
        return 0


In [30]:
# 2-main
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

samplerTest = sample_Z

print(samplerTest(0,1,"D",(4,8)))

print(samplerTest(0,1,"G",(4,8)))

print(torch.normal(mean=0, std=1, size=(4, 8)))

print(samplerTest(0,1,"F",))

tensor([[-0.2370, -0.2052, -0.9498, -1.2684,  0.3710, -1.7311,  0.0395,  0.2506],
        [ 0.4471, -0.4199, -0.7794, -1.7119, -1.0008, -0.2231, -0.3744, -1.8650],
        [ 0.0048, -1.2284,  0.6159, -0.4271, -1.3490, -1.4486, -0.5071, -0.9636],
        [ 0.5744, -0.7888,  0.0239, -0.6671, -0.5900, -0.2174, -0.0077, -0.6219]])
tensor([[-0.3120, -0.1445,  0.3929,  1.4199,  1.1966, -0.7476,  0.7042, -0.0446],
        [ 0.7346, -0.7360, -0.6978, -0.3620, -1.1487, -1.2811, -0.0141, -0.8340],
        [ 0.9923,  1.0578,  0.2753, -0.4284, -0.3946, -0.8002, -2.0094, -2.0477],
        [ 0.2957, -1.8570,  1.5209,  0.9018, -0.0807,  0.3650,  1.8961,  1.5691]])
tensor([[-0.5548, -0.6049,  0.6941, -0.2929, -2.0499, -0.1901, -1.0504,  0.6330],
        [-1.3966, -0.1858,  0.5968,  1.6396,  0.4218, -0.8041,  0.9568, -0.6583],
        [-0.6757, -1.7487,  0.6358, -0.1512, -1.6372, -1.0722, -0.5969,  1.3289],
        [-0.1195,  0.2331, -0.1663,  0.0665,  1.0471,  0.6947, -0.0873, -0.7331]])
0


In [33]:
# Task 3. Train Discriminator
"""
Function to train the Discriminator
"""
import torch


def train_dis(Gen,
              Dis,
              dInputSize,
              gInputSize,
              mbatchSize,
              steps,
              optimizer,
              crit):
    """
    Args:
        Gen, Dis - Generator and Discriminator objects
        dInputSize - Input size of Discriminator input data
        gInputSize - Input size of Generator input data
        mbatchSize - Batch size for training
        steps - number of steps for training
        optimizer - stochastic gradient descent optimizer object
        crit - BCEloss function

    Should use both random noise and normal distribution for sampling
    The 4 moments should be used in processing the sample

    Returns:
        The function should return the error estimate of the fake and real data
        along with the fake and real data sets of type torch.tensor()
    """
    for _ in range(steps):
        # Create real and generated data and labels to train discriminator

        # Use sample_Z function to make some real data points
        reals = sample_Z(mu=0.0,
                         sigma=1.0,
                         sampleType="D",
                         size=(mbatchSize, dInputSize))
        # Make labels for real data, equal to 1
        realLabels = torch.ones((mbatchSize, 1))

        # Use sample_Z function to make some generated data
        generated = sample_Z(mu=0.0,
                             sigma=1.0,
                             sampleType="G",
                             size=(mbatchSize, gInputSize))
        generatedOutput = Gen(generated)
        # Make labels for generated data, equal to 0
        generatedLabels = torch.zeros((mbatchSize, 1))

        # Combine real and generated data/labels to run through discriminator
        data = torch.cat((reals, generated))
        labels = torch.cat((realLabels, generatedLabels))

        # Train the Discriminator on combined data and labels
        Dis.zero_grad()
        output = Dis(data)
        loss = crit(output, labels)
        loss.backward()
        optimizer.step()

    discriminatorOut = Dis(data)
    discriminatorLoss = crit(discriminatorOut, labels)
    generatorOut = Dis(generatedOutput)
    generatorLoss = crit(generatorOut, generatedLabels)

    return generatorLoss, discriminatorLoss, data

In [43]:
# 3-main
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

disTrain = train_dis

learning_rate = 1e-3
sgd_momentum = 0.9
g_input_size = 1   
g_hidden_size = 5  
g_output_size = 1  
minibatch_size =  500 
d_input_size = 1 
d_hidden_size = 10 
d_output_size = 1  
num_epochs = 50
d_steps = 20
g_steps = 20

G_Test = Generator(1,5,1)
D_Test = Discriminator(1,10,1)



opt_Test = optim.SGD(G_Test.parameters(), lr=learning_rate, momentum=sgd_momentum)
crit_Test = nn.BCELoss() 

print(disTrain(G_Test,D_Test, d_input_size,g_input_size, minibatch_size, d_steps, opt_Test, crit_Test))

(tensor(0.7373, grad_fn=<BinaryCrossEntropyBackward0>), tensor(0.6941, grad_fn=<BinaryCrossEntropyBackward0>), tensor([[ 4.9406e-01],
        [-1.1564e+00],
        [ 5.6411e-01],
        [ 1.9861e+00],
        [ 6.6313e-01],
        [ 1.3548e+00],
        [-1.1261e+00],
        [-1.2434e+00],
        [ 7.5872e-01],
        [-6.7988e-01],
        [ 5.3725e-01],
        [-3.8934e-01],
        [ 1.6530e-01],
        [-8.8630e-01],
        [ 2.9453e-01],
        [-6.6635e-01],
        [-1.2349e-01],
        [-7.6418e-01],
        [-3.1607e-01],
        [ 1.9129e+00],
        [-6.8748e-01],
        [-1.4987e+00],
        [-1.1307e+00],
        [ 6.0120e-01],
        [-7.1924e-01],
        [ 1.1517e+00],
        [ 8.3488e-01],
        [-1.6219e-01],
        [ 2.1782e-01],
        [ 7.1012e-01],
        [ 2.3995e-01],
        [-5.4626e-01],
        [-8.2006e-01],
        [ 1.0039e+00],
        [ 6.1838e-01],
        [ 1.2236e+00],
        [ 1.4238e+00],
        [ 9.8173e-01],
        [-8.020